## Imports

In [21]:
import cvxpy as cp
import gurobipy as gp
import numpy as np
import pandas as pd
from gurobipy import GRB

## Read Data

In [22]:
arrivals_df = pd.read_excel("Arrivals.xlsx")
arrivals_df["Type"] = "Arrival"

departures_df = pd.read_excel("Departures.xlsx")
departures_df["Type"] = "Departure"

gates_df = pd.read_excel("Gates.xlsx")

all_flights_df = pd.concat([arrivals_df, departures_df])

In [23]:
arrivals_df

,Flight Number,Arrival Time,Departure Time,Passengers,Type
0,2,25,136,106,Arrival
1,3,41,161,147,Arrival
2,6,73,187,187,Arrival
3,7,77,188,121,Arrival
4,10,111,233,94,Arrival
...,...,...,...,...,...
63,120,1325,1447,127,Arrival
64,124,1369,1499,88,Arrival
65,126,1390,1513,240,Arrival
66,127,1398,1513,87,Arrival


In [24]:
gates_df ## Gates Start from 1 (Gate 0 is the terminal entry)

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0,0,53,45,43,31,29,25,23,19,9,3,15,17,21,23
1,1,53,0,4,10,22,24,28,30,34,44,50,62,64,68,70
2,2,45,4,0,6,18,20,24,26,30,40,46,58,60,64,66
3,3,43,10,6,0,12,14,18,20,24,34,40,52,54,58,60
4,4,31,22,18,12,0,2,6,8,12,22,28,40,42,46,48
5,5,29,24,20,14,2,0,4,6,10,20,26,38,40,44,46
6,6,25,28,24,18,6,4,0,2,6,16,22,34,36,40,42
7,7,23,30,26,20,8,6,2,0,4,14,20,32,34,38,40
8,8,19,34,30,24,12,10,6,4,0,10,16,28,30,34,36
9,9,9,44,40,34,22,20,16,14,10,0,6,18,20,24,26


## Utility Functions

In [25]:
def get_distance(i): # Distance of gate i to gate 0
    return gates_df[0][i]

def get_distance(i, j):
    return gates_df[i][j]

def get_arrival_time(i):
    result = all_flights_df[all_flights_df["Flight Number"] == i]["Arrival Time"]
    assert(len(result) == 1)
    return int(result)

def get_departure_time(i):
    result = all_flights_df[all_flights_df["Flight Number"] == i]["Departure Time"]
    assert(len(result) == 1)
    return int(result)

def get_num_passengers(i):
    result = all_flights_df[all_flights_df["Flight Number"] == i]["Passengers"]
    assert(len(result) == 1)
    return int(result)

### Tests

In [26]:
assert(get_distance(5,3) == 14)
assert(get_distance(14,14) == 0)
assert(get_distance(4,11) == 40)

In [27]:
## Basic Assertions on the Data
flight_numbers = sorted(list(arrivals_df["Flight Number"]) + list(departures_df["Flight Number"]))
assert(flight_numbers == list(range(1,131)))

flight_numbers = sorted(list(all_flights_df["Flight Number"]))
assert(flight_numbers == list(range(1,131)))

## Setup Gates

In [28]:
num_gates = 14
gates = [i for i in range(num_gates)]
gates

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]

## Design Variables

In [29]:
num_flights = 5 # len(flight_numbers)
num_gates = len(gates)
x = cp.Variable((num_flights, len(gates)), boolean=True)

## Constraint 1 (Every aircraft must be assigned to one and only one gate.)

In [30]:
# Every aircraft must be assigned to one and only one gate.
constraint_1_list = []
for i, x_i in enumerate(x):
    constraint_1_list.append(cp.sum(x_i) == 1)

In [31]:
len(constraint_1_list)

5

## Constraint 2 (No two aircrafts can be assigned to the same gate at the same time.)

In [32]:
# No two aircrafts can be assigned to the same gate at the same time.
constraint_2_list = []
for i, x_i in enumerate(x): 
    ## Flight numbers start from 1.
    ai, di = get_arrival_time(i + 1), get_departure_time(i + 1)
    for j, x_j in enumerate(x):
        if i <= j:
            continue
        aj, dj = get_arrival_time(j + 1), get_departure_time(j + 1)
        for k in range(num_gates):
            constraint = (x_i[k]*x_j[k]*(dj - ai)*(di - aj) <= 0)
            constraint_2_list.append(constraint)

In [33]:
len(constraint_2_list)

140

## Define Objective Function

In [41]:
print(x.shape)

(5, 14)


In [42]:
W = np.zeros(x.shape)
P = np.zeros(x.shape)

In [43]:
for i in range(W.shape[0]): # iterate over flights
    for k in range(W.shape[1]): # iterate over gates
        W[i][k] = get_distance(0, k+1) # Gates start from 1
        P[i][k] = get_num_passengers(i+1) # Flights start from 1

In [44]:
objective = cp.sum(cp.multiply(cp.multiply(x, W), P))

In [45]:
# Note: constraint_2_list is a list of non-linear constraints, so we cannot use cvxpy.
all_constraints = constraint_1_list # + constraint_2_list
problem = cp.Problem(cp.Minimize(objective), all_constraints)

## Solve Problem

In [46]:
result = problem.solve(solver=cp.GLPK_MI)

In [47]:
result

2235.0

In [50]:
x.value

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]])

## Gurobi

In [60]:
num_flights = 5
num_gates = 14

In [61]:
F = [f"Flight_{i}" for i in range(1, num_flights + 1)]
F

['Flight_1', 'Flight_2', 'Flight_3', 'Flight_4', 'Flight_5']

In [62]:
G = [f"Gate_{i}" for i in range(1, num_gates + 1)]
G

['Gate_1',
 'Gate_2',
 'Gate_3',
 'Gate_4',
 'Gate_5',
 'Gate_6',
 'Gate_7',
 'Gate_8',
 'Gate_9',
 'Gate_10',
 'Gate_11',
 'Gate_12',
 'Gate_13',
 'Gate_14']

In [63]:
flight_gate_dict = dict()
for f in F:
    for g in G:
        i = int(f.split("_")[1])
        j = int(g.split("_")[1])
        p = get_num_passengers(i)
        w = get_distance(0, j)
        flight_gate_dict[(f,g)] = [p, w]

In [64]:
flight_gate_vars, distances, passengers = gp.multidict(flight_gate_dict)

In [65]:
m = gp.Model('RAP')

Set parameter Username
Academic license - for non-commercial use only - expires 2022-06-28


### Design Variables

In [66]:
# flight_gate_vars = [(f, g) for f in F for g in G]
x = m.addVars(flight_gate_vars, name="assign")

### Constraint 1

In [67]:
constraint_1 = m.addConstrs((x.sum(f,'*') == 1 for f in F), name="constraint_1")

In [68]:
constraint_1

{'Flight_1': <gurobi.Constr *Awaiting Model Update*>,
 'Flight_2': <gurobi.Constr *Awaiting Model Update*>,
 'Flight_3': <gurobi.Constr *Awaiting Model Update*>,
 'Flight_4': <gurobi.Constr *Awaiting Model Update*>,
 'Flight_5': <gurobi.Constr *Awaiting Model Update*>}

### Constraint 2

In [69]:
for f1 in F:
    i = int(f1.split("_")[1])
    ai, di = get_arrival_time(i), get_departure_time(i)
    for f2 in F:
        j = int(f2.split("_")[1])
        if i <= j:
            continue
        aj, dj = get_arrival_time(j), get_departure_time(j)
        for g in G:
            name = f"{f1},{f2},{g}"
            const = m.addConstr((x[(f1, g)] * x[(f2, g)] * (dj - ai) * (di - aj)) <= 0, name=f"{f1},{f2},{g}")

### Objective Function

In [75]:
x

{('Flight_1', 'Gate_1'): <gurobi.Var assign[Flight_1,Gate_1] (value 0.0)>,
 ('Flight_1', 'Gate_2'): <gurobi.Var assign[Flight_1,Gate_2] (value 0.0)>,
 ('Flight_1', 'Gate_3'): <gurobi.Var assign[Flight_1,Gate_3] (value 0.0)>,
 ('Flight_1', 'Gate_4'): <gurobi.Var assign[Flight_1,Gate_4] (value 0.0)>,
 ('Flight_1', 'Gate_5'): <gurobi.Var assign[Flight_1,Gate_5] (value 0.0)>,
 ('Flight_1', 'Gate_6'): <gurobi.Var assign[Flight_1,Gate_6] (value 0.0)>,
 ('Flight_1', 'Gate_7'): <gurobi.Var assign[Flight_1,Gate_7] (value 0.0)>,
 ('Flight_1', 'Gate_8'): <gurobi.Var assign[Flight_1,Gate_8] (value 0.0)>,
 ('Flight_1', 'Gate_9'): <gurobi.Var assign[Flight_1,Gate_9] (value 0.0)>,
 ('Flight_1', 'Gate_10'): <gurobi.Var assign[Flight_1,Gate_10] (value 1.0)>,
 ('Flight_1', 'Gate_11'): <gurobi.Var assign[Flight_1,Gate_11] (value 0.0)>,
 ('Flight_1', 'Gate_12'): <gurobi.Var assign[Flight_1,Gate_12] (value 0.0)>,
 ('Flight_1', 'Gate_13'): <gurobi.Var assign[Flight_1,Gate_13] (value 0.0)>,
 ('Flight_1', 'Ga

In [81]:
objective = 0
for var_name in x.keys():
    xij = x[var_name]
    i = int(var_name[0].split("_")[1])
    j = int(var_name[1].split("_")[1])
    objective += xij * get_num_passengers(i) * get_distance(0, j)

In [82]:
# distances_times_passengers = {k: distances[k]*passengers[k] for k in distances}
# m.setObjective(x.prod(distances_times_passengers), GRB.MINIMIZE)
m.setObjective(objective, GRB.MINIMIZE)

### Optimize

In [83]:
m.params.NonConvex = 2
m.params.MIPGap = 0.1
m.Params.TimeLimit = 600
m.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 5 rows, 70 columns and 70 nonzeros
Model fingerprint: 0x297e3d22
Model has 140 quadratic constraints
Variable types: 70 continuous, 0 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+04, 2e+04]
  Objective range  [3e+02, 1e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]

MIP start from previous solve produced solution with objective 8503 (0.02s)
Loaded MIP start from previous solve with objective 8503

Presolve time: 0.00s
Presolved: 285 rows, 70 columns, 350 nonzeros
Presolved model has 140 bilinear constraint(s)
Variable types: 70 continuous, 0 integer (0 binary)

Root relaxation: objective 4.470000e+03, 5 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf

### Inspect Results

In [72]:
for v in m.getVars():
    if v.x > 1e-6:
        print(v.varName, v.x)

# Display optimal total matching score
print('Total matching score: ', m.objVal)

assign[Flight_1,Gate_10] 1.0
assign[Flight_2,Gate_8] 1.0
assign[Flight_3,Gate_9] 1.0
assign[Flight_4,Gate_12] 1.0
assign[Flight_5,Gate_11] 1.0
Total matching score:  8503.0
